## 新能源电动汽车动力电池充电能量预测（八）——导入数据库

In [1]:
import pandas as pd 
from sklearn import metrics
import numpy as np
from package import utils

### 定义数据处理的类型：1.with_fill（使用填充数据）；2.without_fill（不使用填充数据）

In [2]:
type = 1

### 根据数据库字段导出填充后的数据，方便直接导入数据库

In [3]:
# 数据库需要的字段
header = ['vehicle_id','charge_start_time','charge_end_time','mileage',
          'charge_start_soc','charge_end_soc','charge_start_U','charge_end_U',
          'charge_start_I','charge_end_I','charge_max_temp','charge_min_temp','charge_energy']
# 导入填充后的数据集
# 导入经过填充之后的数据集合
path = 'whole/filled_by_kmeans'
if type == 2:
    path = 'perfect'
dataset = pd.read_csv('./energy_predict_data/data_prehandle/4_fill_null/%s/predict_data_e_train.csv' % path)
dataset_train_test = pd.read_csv('./energy_predict_data/data_prehandle/4_fill_null/whole/filled_by_kmeans/predict_data_e_train.csv')
dataset_train_test = dataset_train_test[header]
# 数据集大小
count = (int)(np.max(dataset_train_test['vehicle_id']))
# 数据描述
pd.set_option('precision', 2)
dataset_train_test.describe()

,vehicle_id,charge_start_time,charge_end_time,mileage,charge_start_soc,charge_end_soc,charge_start_U,charge_end_U,charge_start_I,charge_end_I,charge_max_temp,charge_min_temp,charge_energy
count,5990.00,5.99e+03,5.99e+03,5990.00,5990.00,5990.00,5990.00,5980.00,5990.00,5980.00,5990.00,5990.00,5990.00
mean,11.37,2.02e+13,2.02e+13,103588.11,35.57,76.56,405.94,436.13,-43.74,-22.32,27.19,15.18,25.94
std,6.45,4.00e+09,4.00e+09,30819.13,19.37,26.82,91.53,87.88,48.22,35.36,10.62,11.55,21.24
min,1.00,2.02e+13,2.02e+13,45631.70,0.00,4.80,252.30,0.00,-202.00,-400.00,-2.00,-7.00,0.22
25%,5.00,2.02e+13,2.02e+13,78704.15,20.80,55.25,340.40,374.60,-59.80,-20.40,18.00,5.00,12.64
50%,11.00,2.02e+13,2.02e+13,97276.05,34.00,87.00,361.50,391.60,-24.00,-9.30,30.00,14.00,19.61
75%,17.00,2.02e+13,2.02e+13,122780.75,47.00,100.00,533.70,551.70,-9.00,-5.00,35.00,25.00,29.56
max,21.00,2.02e+13,2.02e+13,182536.00,96.00,110.49,574.60,604.70,-0.10,8.00,52.00,46.00,103.72


#### 将时间字符串转化为时间戳

In [4]:
dataset_train_test['charge_start_time'] = np.array(utils.calVecTimeStamp(dataset_train_test['charge_start_time']))
dataset_train_test['charge_end_time'] = np.array(utils.calVecTimeStamp(dataset_train_test['charge_end_time']))

### 导出预测表，注意外键的限制。

#### 先获取外键

In [5]:
ref_key_list = []
for i in range(1, count+1):
    data_vehicle = dataset_train_test[dataset_train_test['vehicle_id'] == i]
    index_list = data_vehicle.index.tolist()
    ref_key = index_list[int(0.75*len(data_vehicle)):len(data_vehicle)]
    for i in range(len(ref_key)):
        ref_key_list.append(ref_key[i]+1)

#### 导入预测数据

In [6]:
dataset_predict = pd.read_csv('./energy_predict_data/predict_result/data_pred_%s.csv' % utils.getType(type))
head_pred = dataset_predict.columns.values.tolist()[1:]
# 数据描述
pd.set_option('precision', 2)
dataset_predict.describe()

,vehicle_id,type,lr,svr_lr,svr_rbf,dtr,rtr,etr,abr,br,gbr,xgbr,mlpr
count,1506.00,1506.0,1506.00,1506.00,1506.00,1506.00,1506.00,1506.00,1506.00,1506.00,1506.00,1506.00,1506.00
mean,11.37,1.0,24.82,24.05,23.88,25.31,25.44,25.62,27.06,25.51,25.36,25.37,25.42
std,6.45,0.0,21.83,19.09,17.61,21.94,21.78,21.92,19.75,21.88,21.69,21.76,21.79
min,1.00,1.0,-15.19,-7.89,-1.49,0.22,0.44,0.45,9.78,0.38,-0.24,-0.12,-1.82
25%,5.00,1.0,10.59,11.56,12.21,11.51,12.01,12.22,14.73,12.09,12.09,12.06,11.65
50%,11.00,1.0,19.67,18.67,18.40,18.71,18.87,19.14,19.08,19.10,18.86,18.71,19.10
75%,17.00,1.0,32.85,30.19,29.27,28.97,28.60,28.75,30.08,28.45,28.16,28.17,28.20
max,21.00,1.0,84.08,77.71,74.17,102.70,102.36,99.75,92.77,102.27,100.17,101.33,101.12


In [7]:
print(head_pred)

['type', 'lr', 'svr_lr', 'svr_rbf', 'dtr', 'rtr', 'etr', 'abr', 'br', 'gbr', 'xgbr', 'mlpr']


#### 预测表所需字段

In [8]:
columns_pred = ['id', 'type', 'lr', 'plr', 'svr_lr', 'svr_plr',
           'svr_rbf', 'dtr', 'rtr', 'etr', 'abr', 'br', 
           'gbr', 'xgbr', 'mlpr', 'bpr']

#### 构造预测表的数据

In [9]:
mysql_pred = pd.DataFrame(columns=columns_pred)
for col in head_pred:
    mysql_pred[col] = dataset_predict[col]
for col in list(set(columns_pred).difference(set(head_pred))):
    mysql_pred[col] = [np.NaN] * len(dataset_predict)
mysql_pred['id'] = np.array(ref_key_list)
mysql_pred.describe()

,id,type,lr,plr,svr_lr,svr_plr,svr_rbf,dtr,rtr,etr,abr,br,gbr,xgbr,mlpr,bpr
count,1506.00,1506.0,1506.00,0.0,1506.00,0.0,1506.00,1506.00,1506.00,1506.00,1506.00,1506.00,1506.00,1506.00,1506.00,0.0
mean,3127.79,1.0,24.82,NaN,24.05,NaN,23.88,25.31,25.44,25.62,27.06,25.51,25.36,25.37,25.42,NaN
std,1726.21,0.0,21.83,NaN,19.09,NaN,17.61,21.94,21.78,21.92,19.75,21.88,21.69,21.76,21.79,NaN
min,164.00,1.0,-15.19,NaN,-7.89,NaN,-1.49,0.22,0.44,0.45,9.78,0.38,-0.24,-0.12,-1.82,NaN
25%,1666.25,1.0,10.59,NaN,11.56,NaN,12.21,11.51,12.01,12.22,14.73,12.09,12.09,12.06,11.65,NaN
50%,3016.50,1.0,19.67,NaN,18.67,NaN,18.40,18.71,18.87,19.14,19.08,19.10,18.86,18.71,19.10,NaN
75%,4596.75,1.0,32.85,NaN,30.19,NaN,29.27,28.97,28.60,28.75,30.08,28.45,28.16,28.17,28.20,NaN
max,5990.00,1.0,84.08,NaN,77.71,NaN,74.17,102.70,102.36,99.75,92.77,102.27,100.17,101.33,101.12,NaN


### 保存文件

In [10]:
dataset_train_test.to_csv('./energy_predict_data/data_2_mysql/dmpj_train_test_data.csv', header=None, index = False)
mysql_pred.to_csv('./energy_predict_data/data_2_mysql/dmpj_predict_data.csv', header=None, index = False)